In [18]:
import sys
import os
import pandas as pd
sys.path.append("../src")
import json
from data import data
from constants import Timeframe
from datetime import datetime, timedelta
from pydantic import BaseModel, ValidationError, PositiveInt
from datetime import datetime
from datetime import timedelta
from backtesting.backtest_config import BacktestConfig
from pydantic import BaseModel
from api.coindesk_api import get_OHLC

pd.set_option('display.max_columns', None)

In [10]:
from_date = datetime(2022, 1, 1)
to_date = datetime.now().replace(microsecond=0, second=0, minute=0)
tick_limit=2000

def fetch_data(from_dt: datetime, to_dt: datetime):
    # This is your second function that gets called for each chunk
    # Replace this with actual logic (e.g., API calls, DB queries, etc.)
    print(f"Fetching from {from_dt} to {to_dt}")
    return [(from_dt, to_dt)]  # Dummy return for example

def chunked_fetch(start: datetime, end: datetime):
    chunk_size = timedelta(hours=2000)
    results = []

    current = start
    while current < end:
        chunk_end = min(current + chunk_size, end)
        chunk_result = fetch_data(current, chunk_end)
        results.extend(chunk_result)
        current = chunk_end

    return results

result = chunked_fetch(start=from_date, end=to_date)

print(result)

Fetching from 2022-01-01 00:00:00 to 2022-03-25 08:00:00
Fetching from 2022-03-25 08:00:00 to 2022-06-16 16:00:00
Fetching from 2022-06-16 16:00:00 to 2022-09-08 00:00:00
Fetching from 2022-09-08 00:00:00 to 2022-11-30 08:00:00
Fetching from 2022-11-30 08:00:00 to 2023-02-21 16:00:00
Fetching from 2023-02-21 16:00:00 to 2023-05-16 00:00:00
Fetching from 2023-05-16 00:00:00 to 2023-08-07 08:00:00
Fetching from 2023-08-07 08:00:00 to 2023-10-29 16:00:00
Fetching from 2023-10-29 16:00:00 to 2024-01-21 00:00:00
Fetching from 2024-01-21 00:00:00 to 2024-04-13 08:00:00
Fetching from 2024-04-13 08:00:00 to 2024-07-05 16:00:00
Fetching from 2024-07-05 16:00:00 to 2024-09-27 00:00:00
Fetching from 2024-09-27 00:00:00 to 2024-12-19 08:00:00
Fetching from 2024-12-19 08:00:00 to 2025-03-12 16:00:00
Fetching from 2025-03-12 16:00:00 to 2025-06-04 00:00:00
Fetching from 2025-06-04 00:00:00 to 2025-07-16 05:00:00
[(datetime.datetime(2022, 1, 1, 0, 0), datetime.datetime(2022, 3, 25, 8, 0)), (datetime.

In [19]:
df = data.get_df("ADA_USD", Timeframe.H1, ".", 'PARQUET')
df.head(10)

,timestamp,open,high,low,close
0,01-01-2021 00:00,0.182239,0.183608,0.182239,0.183417
1,01-01-2021 01:00,0.183417,0.183457,0.180281,0.181469
2,01-01-2021 02:00,0.181469,0.181469,0.178876,0.180049
3,01-01-2021 03:00,0.180049,0.180489,0.177549,0.178461
4,01-01-2021 04:00,0.178461,0.179856,0.178340,0.179453
5,01-01-2021 05:00,0.179453,0.180735,0.179030,0.180653
6,01-01-2021 06:00,0.180653,0.180982,0.179862,0.180092
7,01-01-2021 07:00,0.180092,0.182549,0.179877,0.180786
8,01-01-2021 08:00,0.180786,0.181961,0.180786,0.181595
9,01-01-2021 09:00,0.181595,0.183802,0.181117,0.183394


In [22]:
class Trade(BaseModel):
    # price of quote crypto
    open_price: float = 0.0
    # total value of quote crypto in base
    open_value: float = 0.0
    # trade size of base crypto/currency(USD)
    total_quantity: float = 0.0
    # price of quote crypto
    current_price: float = 0.0
    # profit/loss in base crypto/currency(USD)
    profit_loss: float = 0.0
    # profit/loss percentage
    profit_loss_pct: float = 0.0
    # value of quote crypto in base
    current_value: float = 0.0
    averaging_orders_prices: list = []
    averaging_order_sizes: list = []
    # list of the averaging order quantities (order_size / price), how many of the quote did you get
    averaging_orders_qty: list = []
    averaging_orders_filled: int = 0
    last_order_price: float = 0.0

def calc_avgeraging_order_prices(initial_price:float, deviation:float, multiplier:float):
        prices = []
        cumulative_deviation = 0

        for i in range(6):
            cumulative_deviation += deviation
            price = initial_price * (1 - (cumulative_deviation / 100))
            prices.append(round(price, 7))
        return prices

def calc_averaging_order_size(avg_order_size: float, multiplier: float):
     sizes = [
          round(avg_order_size * (multiplier ** i), 2)
          for i in range(6)
     ]
     return sizes

def calc_profit_losses(current_price: float, total_cost: float, total_qty: float):
    current_value = current_price * total_qty
    profit_loss = current_value - total_cost
    profit_loss_pct = (profit_loss / total_cost) * 100

    return round(profit_loss, 6), round(profit_loss_pct, 6)
     

def run_backtest(df):
    df_copy = df.copy()
    df_copy['trade_action'] = 'no_trade'
    df_copy['trade_value'] = 'None'
    df_copy['trade_profit_loss'] = 0.0
    df_copy['trade_profit_loss_pct'] = 0.0

    trades: list[Trade] = []

    current_trade = Trade()

    total_profit = 0.0
    hit_take_profit = False
    
    #bot config
    base_order_size = 20
    averaging_order_size = 40
    max_averaging_orders = 6
    take_profit = 1.5
    price_deviation = 1.5
    averaging_order_size_mul = 1.28
    averaging_order_step_mul = 1.85

    #account info
    starting_account_balance = 500.0
    current_account_balance = 500.0

    for index, row in enumerate(df_copy.itertuples(), 0):
        if current_trade.profit_loss_pct > take_profit:
            total_profit += current_trade.profit_loss
            df_copy.loc[index, 'trade_action'] = 'closed_trade'
            df_copy.loc[index + 1, 'trade_action'] = 'no_trade'
            current_trade = Trade()
        elif row.trade_action == 'in_trade':
            # maintain trade
            avg_orders_filled = current_trade.averaging_orders_filled
            if float(row.low) < current_trade.averaging_orders_prices[avg_orders_filled] and avg_orders_filled < 5:
                 current_trade.last_order_price = current_trade.averaging_orders_prices[avg_orders_filled]
                 current_trade.total_quantity += (current_trade.averaging_order_sizes[avg_orders_filled] / float(row.low))
                 current_trade.open_value += current_trade.averaging_order_sizes[avg_orders_filled]

                 df_copy.loc[index, 'trade_action'] = f"avg. order {avg_orders_filled + 1} filled"
                 if (avg_orders_filled < 5): current_trade.averaging_orders_filled += 1
            current_trade.current_value = float(current_trade.total_quantity * float(row.open))
            
            # current_price_deviation = ((float(row.open) - current_trade.last_order_price) / current_trade.last_order_price) * 100

            profit_loss, profit_loss_pct = calc_profit_losses(
                 current_price=float(row.open), 
                 total_cost=current_trade.open_value, 
                 total_qty=current_trade.total_quantity
                 )
            current_trade.profit_loss = profit_loss
            current_trade.profit_loss_pct = profit_loss_pct

            df_copy.loc[index, 'trade_value'] = current_trade.current_value
            df_copy.loc[index, 'trade_profit_loss'] = current_trade.profit_loss
            df_copy.loc[index, 'trade_profit_loss_pct'] = current_trade.profit_loss_pct
            df_copy.loc[index + 1, 'trade_action'] = 'in_trade'

        elif row.trade_action == 'no_trade':
            # open trade
            current_trade.total_quantity = base_order_size / float(row.open)
            current_trade.open_value = float(current_trade.total_quantity * float(row.open))
            current_trade.last_order_price = float(row.open)
            current_trade.averaging_orders_prices = calc_avgeraging_order_prices(
                initial_price=float(row.open),
                deviation=price_deviation,
                multiplier=averaging_order_step_mul
                )
            current_trade.averaging_order_sizes = calc_averaging_order_size(
                 avg_order_size=averaging_order_size,
                 multiplier=averaging_order_size_mul
            )
            
            df_copy.loc[index, 'trade_action'] = 'opened_trade'
            df_copy.loc[index, 'trade_value'] = current_trade.open_value
            df_copy.loc[index + 1, 'trade_action'] = 'in_trade'
    trades.append(current_trade)

    return df_copy, trades, total_profit

In [23]:
df_tested, trades, profit = run_backtest(df)
print(profit)
df_tested.head(60)

779.5892250000007


,timestamp,open,high,low,close,trade_action,trade_value,trade_profit_loss,trade_profit_loss_pct
0,01-01-2021 00:00,0.182239,0.183608,0.182239,0.183417,opened_trade,20.0,0.000000,0.000000
1,01-01-2021 01:00,0.183417,0.183457,0.180281,0.181469,in_trade,20.129334,0.129334,0.646669
2,01-01-2021 02:00,0.181469,0.181469,0.178876,0.180049,avg. order 1 filled,60.495285,0.495285,0.825474
3,01-01-2021 03:00,0.180049,0.180489,0.177549,0.178461,in_trade,60.021876,0.021876,0.036460
4,01-01-2021 04:00,0.178461,0.179856,0.178340,0.179453,in_trade,59.492394,-0.507606,-0.846011
5,01-01-2021 05:00,0.179453,0.180735,0.179030,0.180653,in_trade,59.823336,-0.176664,-0.294439
6,01-01-2021 06:00,0.180653,0.180982,0.179862,0.180092,in_trade,60.223262,0.223262,0.372104
7,01-01-2021 07:00,0.180092,0.182549,0.179877,0.180786,in_trade,60.036314,0.036314,0.060524
8,01-01-2021 08:00,0.180786,0.181961,0.180786,0.181595,in_trade,60.267708,0.267708,0.446179
9,01-01-2021 09:00,0.181595,0.183802,0.181117,0.183394,in_trade,60.537204,0.537204,0.895340


In [ ]:
%%timeit
run_backtest(df)

# Testing Ways to Loop or Vectorize DF:

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def get_data(size = 10_000):
    df = pd.DataFrame()
    df['age'] = np.random.randint(0, 100, size)
    df['time_in_bed'] = np.random.randint(0, 9, size)
    df['pct_sleeping'] = np.random.rand(size)
    df['favorite_food'] = np.random.choice(['pizza', 'taco', 'ice-cream'], size)
    df['hate_food'] = np.random.choice(['broccoli', 'candy corn', 'eggs'], size)
    return df

## The problem
Reqard calc:
- If they were in bed for more than 5 hours AND they were sleeping for more than 50% we give them their favorite food.
- Otherwise we give them their hate food.
- If they are over 90 years old give their favorite foor regardless

In [ ]:
def reward_calc(row):
    if row['age'] >= 90:
        return row['favorite_food']
    if (row['time_in_bed'] > 5) & (row['pct_sleeping'] > 0.5):
        return row['favorite_food']
    return row['hate_food']

### Level 1 - Loop (Slowest)

In [ ]:
df = get_data()

In [ ]:
%%timeit
for index, row in df.iterrows():
    df.loc[index, 'reward'] = reward_calc(row)

### Level 2 - Apply

In [ ]:
df = get_data()

In [ ]:
%%timeit
df['reward'] = df.apply(reward_calc, axis=1)

### Level 3 - Vectorized (Fastest)

In [ ]:
df = get_data()

In [ ]:
%%timeit
df['reward'] = df['hate_food']
df.loc[
    ((df['pct_sleeping'] > 0.5) & (df['time_in_bed'] > 5) | (df['age'] > 90)), 
    'reward'
    ] = df['favorite_food']